# DATA WRANGLING
___________________________________________________________________________________________________________________

## 1. Case study question:
### Use data wrangling tools to calculate the total revenue of each product at each price based on 02 sample datasets:

- price.csv: records of product's history price changes in Sep 2018
- sale.csv: records of product's sale in Sep 2018

## 2. Solution:


### a. Using `merge_asof()`

In [1]:
#import necessary package

import pandas as pd
import numpy as np

In [2]:
#import file

prices_df = pd.read_csv('prices.csv')
sales_df = pd.read_csv('sales.csv')

In [3]:
#check file: no null values

prices_df.info()
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  14 non-null     int64 
 1   old_price   14 non-null     int64 
 2   new_price   14 non-null     int64 
 3   updated_at  14 non-null     object
dtypes: int64(3), object(1)
memory usage: 576.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB


In [4]:
#convert ordered_at, updated_at to datetime object (using pd.to_datetime)

prices_df['updated_at'] = pd.to_datetime(prices_df['updated_at'])
sales_df['ordered_at'] = pd.to_datetime(sales_df['ordered_at'])

In [5]:
prices_df = prices_df.sort_values('updated_at')
prices_df

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


In [6]:
sales_df = sales_df.sort_values('ordered_at')
sales_df

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


### With direction='nearest'

In [7]:
#find the "nearest" update timestamp of each order

merged_df = pd.merge_asof(sales_df, prices_df, left_on='ordered_at', right_on='updated_at', 
                          by='product_id', direction = 'nearest')

In [8]:
#compare ordered_at and "nearest" updated_at of each order to find the correct price

merged_df['final_price'] = np.where(merged_df['ordered_at'] >= merged_df['updated_at'], 
                                    merged_df['new_price'], merged_df['old_price'])
merged_df.head(10)

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,final_price
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
5,4085861,2018-09-11 11:06:00,1,60000,53500,2018-09-11 08:51:00,53500
6,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
7,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
8,4085861,2018-09-11 11:34:00,1,60000,53500,2018-09-11 08:51:00,53500
9,4085861,2018-09-11 11:47:00,2,60000,53500,2018-09-11 08:51:00,53500


In [9]:
#calculate total revenue for each product at each price

merged_df['revenue'] = merged_df['final_price'] * merged_df['quantity_ordered']
revenue_df = merged_df.groupby(['product_id','final_price'],as_index=False)['revenue'].sum()
revenue_df

,product_id,final_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [10]:
#calculate total revenue for each product

total_revenue = merged_df.groupby('product_id',as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500


In [14]:
# Alternative to np.where: create a function with if-else condition

def price_nr_1(row):
    if row['ordered_at'] >= row['updated_at']:
        return row['new_price']
    else:
        return row['old_price']

merged_df['price1'] = merged_df.apply(price_nr_1, axis=1)
merged_df

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,final_price,revenue,price1
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000,60000,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000,60000,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000,60000,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000,60000,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500,53500,53500
...,...,...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00,52000,52000,52000
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00,52000,52000,52000
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00,52000,52000,52000
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00,57500,57500,57500


In [15]:
merged_df.to_csv('final.csv')

In [16]:
merged_df['revenue1'] = merged_df['quantity_ordered'] * merged_df['price1']
total_revenue1_by_product = merged_df.groupby(['product_id'], as_index=False)['revenue1'].sum()
total_revenue1_by_product

,product_id,revenue1
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500


### With direction='backward' (combining with direction='forward')

In [20]:
# merge sales and prices table with direction='backward' (default value, which can be optionally omitted)

merge_backward = pd.merge_asof(sales_df, prices_df, by='product_id', left_on='ordered_at', right_on='updated_at', direction='backward')
merge_backward = merge_backward[['product_id', 'ordered_at', 'quantity_ordered', 'new_price']].rename(columns={'new_price':'price_bw'})
merge_backward.head(10)

,product_id,ordered_at,quantity_ordered,price_bw
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,NaN
2,4085861,2018-09-11 06:53:00,1,NaN
3,4085861,2018-09-11 08:24:00,1,NaN
4,4085861,2018-09-11 09:30:00,1,53500.0
5,4085861,2018-09-11 11:06:00,1,53500.0
6,3954203,2018-09-11 11:11:00,1,60000.0
7,3954203,2018-09-11 11:11:00,1,60000.0
8,4085861,2018-09-11 11:34:00,1,53500.0
9,4085861,2018-09-11 11:47:00,2,53500.0


In [22]:
# merge merge_backward and prices tables with direction='forward'

merge_forward = pd.merge_asof(merge_backward, prices_df, by='product_id', left_on='ordered_at', right_on='updated_at', direction='forward')
merge_forward.head(10)

,product_id,ordered_at,quantity_ordered,price_bw,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,60000.0,60000.0,64000.0,2018-09-11 11:54:00
1,4085861,2018-09-11 06:26:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,53500.0,53500.0,67000.0,2018-09-12 03:51:00
5,4085861,2018-09-11 11:06:00,1,53500.0,53500.0,67000.0,2018-09-12 03:51:00
6,3954203,2018-09-11 11:11:00,1,60000.0,60000.0,64000.0,2018-09-11 11:54:00
7,3954203,2018-09-11 11:11:00,1,60000.0,60000.0,64000.0,2018-09-11 11:54:00
8,4085861,2018-09-11 11:34:00,1,53500.0,53500.0,67000.0,2018-09-12 03:51:00
9,4085861,2018-09-11 11:47:00,2,53500.0,53500.0,67000.0,2018-09-12 03:51:00


In [23]:
# fill null values in 'price' column by 'old_price' column where necessary

merge_forward['final_price'] = merge_forward['price_bw'].fillna(merge_forward['old_price'])
final_data = merge_forward[['product_id', 'ordered_at', 'quantity_ordered', 'final_price']]
final_data

,product_id,ordered_at,quantity_ordered,final_price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,60000.0
2,4085861,2018-09-11 06:53:00,1,60000.0
3,4085861,2018-09-11 08:24:00,1,60000.0
4,4085861,2018-09-11 09:30:00,1,53500.0
...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,52000.0
171,4085861,2018-09-18 20:43:00,1,52000.0
172,4085861,2018-09-18 20:54:00,1,52000.0
173,3954203,2018-09-18 21:26:00,1,57500.0


In [24]:
final_data['revenue'] = final_data['quantity_ordered'] * final_data['final_price']
revenue_by_product_and_price = final_data.groupby(['product_id', 'final_price'], as_index=False)['revenue'].sum()
revenue_by_product_and_price

/var/folders/v8/nyfc3kjn3kd6jh02f956qvxm0000gn/T/ipykernel_29178/293877036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['revenue'] = final_data['quantity_ordered'] * final_data['final_price']


,product_id,final_price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [25]:
total_revenue = final_data.groupby('product_id', as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0


## b. Use regular `merge()`

In [27]:
# find quantity of orders of each product at each timestamp

sales_quant = sales_df.groupby(['product_id', 'ordered_at'])['quantity_ordered'].sum().reset_index()
sales_quant

,product_id,ordered_at,quantity_ordered
0,64,2018-09-11 19:32:00,1
1,64,2018-09-12 13:23:00,1
2,64,2018-09-12 16:00:00,1
3,64,2018-09-14 20:10:00,1
4,3954203,2018-09-11 01:43:00,1
...,...,...,...
168,4085861,2018-09-18 15:56:00,1
169,4085861,2018-09-18 18:52:00,1
170,4085861,2018-09-18 20:23:00,1
171,4085861,2018-09-18 20:43:00,1


In [29]:
# merge grouped dataframe to original prices dataframe

merge = pd.merge(sales_quant, prices_df, on='product_id', how='left')
merge

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,64,2018-09-11 19:32:00,1,270000,239000,2018-09-10 16:37:00
1,64,2018-09-12 13:23:00,1,270000,239000,2018-09-10 16:37:00
2,64,2018-09-12 16:00:00,1,270000,239000,2018-09-10 16:37:00
3,64,2018-09-14 20:10:00,1,270000,239000,2018-09-10 16:37:00
4,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00
...,...,...,...,...,...,...
939,4085861,2018-09-18 20:54:00,1,53500,67000,2018-09-12 03:51:00
940,4085861,2018-09-18 20:54:00,1,67000,62500,2018-09-13 06:43:00
941,4085861,2018-09-18 20:54:00,1,62500,58000,2018-09-15 03:51:00
942,4085861,2018-09-18 20:54:00,1,58000,53500,2018-09-17 03:35:00


In [30]:
# find distance between ordered_at and updated_at of each order

merge['time_diff'] = abs(merge['ordered_at'] - merge['updated_at'])
merge

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,time_diff
0,64,2018-09-11 19:32:00,1,270000,239000,2018-09-10 16:37:00,1 days 02:55:00
1,64,2018-09-12 13:23:00,1,270000,239000,2018-09-10 16:37:00,1 days 20:46:00
2,64,2018-09-12 16:00:00,1,270000,239000,2018-09-10 16:37:00,1 days 23:23:00
3,64,2018-09-14 20:10:00,1,270000,239000,2018-09-10 16:37:00,4 days 03:33:00
4,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,0 days 09:11:00
...,...,...,...,...,...,...,...
939,4085861,2018-09-18 20:54:00,1,53500,67000,2018-09-12 03:51:00,6 days 17:03:00
940,4085861,2018-09-18 20:54:00,1,67000,62500,2018-09-13 06:43:00,5 days 14:11:00
941,4085861,2018-09-18 20:54:00,1,62500,58000,2018-09-15 03:51:00,3 days 17:03:00
942,4085861,2018-09-18 20:54:00,1,58000,53500,2018-09-17 03:35:00,1 days 17:19:00


In [31]:
# choose the order with the minimum time difference (similar to merge_asof(direction='nearest'))

merge_group = merge.groupby(['product_id', 'ordered_at', 'quantity_ordered'])['time_diff'].min().reset_index()
merge_group

,product_id,ordered_at,quantity_ordered,time_diff
0,64,2018-09-11 19:32:00,1,1 days 02:55:00
1,64,2018-09-12 13:23:00,1,1 days 20:46:00
2,64,2018-09-12 16:00:00,1,1 days 23:23:00
3,64,2018-09-14 20:10:00,1,4 days 03:33:00
4,3954203,2018-09-11 01:43:00,1,0 days 09:11:00
...,...,...,...,...
168,4085861,2018-09-18 15:56:00,1,0 days 16:57:00
169,4085861,2018-09-18 18:52:00,1,0 days 19:53:00
170,4085861,2018-09-18 20:23:00,1,0 days 21:24:00
171,4085861,2018-09-18 20:43:00,1,0 days 21:44:00


In [32]:
final_merge = pd.merge(merge_group, merge, how='left', on=['product_id', 'ordered_at', 'time_diff']).reset_index()
final_merge

,index,product_id,ordered_at,quantity_ordered_x,time_diff,quantity_ordered_y,old_price,new_price,updated_at
0,0,64,2018-09-11 19:32:00,1,1 days 02:55:00,1,270000,239000,2018-09-10 16:37:00
1,1,64,2018-09-12 13:23:00,1,1 days 20:46:00,1,270000,239000,2018-09-10 16:37:00
2,2,64,2018-09-12 16:00:00,1,1 days 23:23:00,1,270000,239000,2018-09-10 16:37:00
3,3,64,2018-09-14 20:10:00,1,4 days 03:33:00,1,270000,239000,2018-09-10 16:37:00
4,4,3954203,2018-09-11 01:43:00,1,0 days 09:11:00,1,68800,60000,2018-09-10 16:32:00
...,...,...,...,...,...,...,...,...,...
168,168,4085861,2018-09-18 15:56:00,1,0 days 16:57:00,1,53500,52000,2018-09-17 22:59:00
169,169,4085861,2018-09-18 18:52:00,1,0 days 19:53:00,1,53500,52000,2018-09-17 22:59:00
170,170,4085861,2018-09-18 20:23:00,1,0 days 21:24:00,1,53500,52000,2018-09-17 22:59:00
171,171,4085861,2018-09-18 20:43:00,1,0 days 21:44:00,1,53500,52000,2018-09-17 22:59:00


In [33]:
final_merge['correct_price'] = np.where(final_merge['ordered_at'] >= final_merge['updated_at'],
                                        final_merge['new_price'], final_merge['old_price'])
final_merge

,index,product_id,ordered_at,quantity_ordered_x,time_diff,quantity_ordered_y,old_price,new_price,updated_at,correct_price
0,0,64,2018-09-11 19:32:00,1,1 days 02:55:00,1,270000,239000,2018-09-10 16:37:00,239000
1,1,64,2018-09-12 13:23:00,1,1 days 20:46:00,1,270000,239000,2018-09-10 16:37:00,239000
2,2,64,2018-09-12 16:00:00,1,1 days 23:23:00,1,270000,239000,2018-09-10 16:37:00,239000
3,3,64,2018-09-14 20:10:00,1,4 days 03:33:00,1,270000,239000,2018-09-10 16:37:00,239000
4,4,3954203,2018-09-11 01:43:00,1,0 days 09:11:00,1,68800,60000,2018-09-10 16:32:00,60000
...,...,...,...,...,...,...,...,...,...,...
168,168,4085861,2018-09-18 15:56:00,1,0 days 16:57:00,1,53500,52000,2018-09-17 22:59:00,52000
169,169,4085861,2018-09-18 18:52:00,1,0 days 19:53:00,1,53500,52000,2018-09-17 22:59:00,52000
170,170,4085861,2018-09-18 20:23:00,1,0 days 21:24:00,1,53500,52000,2018-09-17 22:59:00,52000
171,171,4085861,2018-09-18 20:43:00,1,0 days 21:44:00,1,53500,52000,2018-09-17 22:59:00,52000


In [37]:
final_merge['correct_price'] = np.where(final_merge['ordered_at'] >= final_merge['updated_at'],
                                        final_merge['new_price'], final_merge['old_price'])
final_merge

,index,product_id,ordered_at,quantity_ordered_x,time_diff,quantity_ordered_y,old_price,new_price,updated_at,correct_price
0,0,64,2018-09-11 19:32:00,1,1 days 02:55:00,1,270000,239000,2018-09-10 16:37:00,239000
1,1,64,2018-09-12 13:23:00,1,1 days 20:46:00,1,270000,239000,2018-09-10 16:37:00,239000
2,2,64,2018-09-12 16:00:00,1,1 days 23:23:00,1,270000,239000,2018-09-10 16:37:00,239000
3,3,64,2018-09-14 20:10:00,1,4 days 03:33:00,1,270000,239000,2018-09-10 16:37:00,239000
4,4,3954203,2018-09-11 01:43:00,1,0 days 09:11:00,1,68800,60000,2018-09-10 16:32:00,60000
...,...,...,...,...,...,...,...,...,...,...
168,168,4085861,2018-09-18 15:56:00,1,0 days 16:57:00,1,53500,52000,2018-09-17 22:59:00,52000
169,169,4085861,2018-09-18 18:52:00,1,0 days 19:53:00,1,53500,52000,2018-09-17 22:59:00,52000
170,170,4085861,2018-09-18 20:23:00,1,0 days 21:24:00,1,53500,52000,2018-09-17 22:59:00,52000
171,171,4085861,2018-09-18 20:43:00,1,0 days 21:44:00,1,53500,52000,2018-09-17 22:59:00,52000


In [38]:
final_merge['revenue'] = final_merge['quantity_ordered_x'] * final_merge['correct_price']

total_revenue = final_merge.groupby(['product_id'])['revenue'].sum().reset_index()
total_revenue

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500


## c. Use `def` function

In [48]:
#def function that return the closest date that price update
def closest_update_date (id, order_date):
    new_date = prices_df['updated_at'][prices_df['product_id'] == id]
    if order_date < new_date.min():
       closest_date = new_date.min()
    else :
       closest_date = new_date[order_date > new_date].max()
    return closest_date

In [49]:
#def function that return closest price base on productid and order_date
def return_closest_price(id, order_date):
    day = closest_update_date (id, order_date)
    if order_date < day :
       closest_price = prices_df['old_price'][(prices_df['updated_at'] == day) & (prices_df['product_id'] == id)].values[0]
    else :
       closest_price = prices_df['new_price'][(prices_df['updated_at'] == day) & (prices_df['product_id'] == id)].values[0]
    return closest_price

In [50]:
#Find price corresponding each row and append to update_price array.
update_price=[]
for i in np.arange(len(sales_df)):
    updated_price = return_closest_price(sales_df.iloc[i]['product_id'], sales_df.iloc[i]['ordered_at'])
    update_price.append(updated_price)

#create and calculate price and revenue column
sales_df["price"] = update_price
sales_df["revenue"] = sales_df['price'] * sales_df['quantity_ordered']

#export to csv file
sales_df
# sale.to_csv("revenue.csv")

,product_id,ordered_at,quantity_ordered,price,revenue
86,3954203,2018-09-11 01:43:00,1,60000,60000
28,4085861,2018-09-11 06:26:00,1,60000,60000
26,4085861,2018-09-11 06:53:00,1,60000,60000
27,4085861,2018-09-11 08:24:00,1,60000,60000
123,4085861,2018-09-11 09:30:00,1,53500,53500
...,...,...,...,...,...
67,4085861,2018-09-18 20:23:00,1,52000,52000
77,4085861,2018-09-18 20:43:00,1,52000,52000
79,4085861,2018-09-18 20:54:00,1,52000,52000
87,3954203,2018-09-18 21:26:00,1,57500,57500


In [52]:
sales_df.groupby('product_id').agg({'revenue':sum})

,revenue
product_id,
64,956000
3954203,877500
3998909,280500
4085861,8247500
